# Importation

In [2]:
import mysql.connector as connection
import pandas as pd
from datetime import datetime

# paramètre de connexion à la base de données

In [3]:
host = "localhost"
port = "3306"
database = "my_data_base"
user = "root"
passwd="pwd"
base_url = "/tmp"

# Chargement des données

In [4]:
def load_data(query):
    try:
        mydb = connection.connect(host=host,port=port, database = database,
                                  user=user, passwd=passwd,use_pure=True)
        result_dataFrame = pd.read_sql(query,mydb)
        mydb.close()
        return result_dataFrame
    except Exception as e:
        print(str(e))

# Sauvegarde des données dans un fichier csv

In [6]:
def save_data(query,filename):
    data = load_data(query)
    data.to_csv(filename,sep=",",index=False,encoding='utf-8')

# Tableau des commandes non livrés

Il faudra fourni toutes les commandes finalisées et non livrées associées à la date fourni. Ainsi que les informations sur le client et la livraison

## Requête

Date à associer à la requête.

In [7]:
date_cmd_non_livre = "2023-06-12"

In [11]:
comande_non_livre_utilisateur_info = 'SELECT cmd.id, cmd.token_value,cmd.number AS numero,cmd.checkout_completed_at AS Date_finalisation_commande,cmd.items_total,cmd.adjustments_total AS Frais_livraison, cmd.total,billingadress.first_name AS Facturation_first_name,billingadress.last_name AS Facturation_last_name,billingadress.phone_number as Facturation_phone_number, shipadress.first_name AS Livraison_first_name,shipadress.last_name AS Livraison_last_name,shipadress.phone_number as Livraison_phone_number,cmd.shipping_state FROM andaal_db.sylius_order AS cmd JOIN andaal_db.sylius_address AS billingadress  ON cmd.billing_address_id = billingadress.id  JOIN andaal_db.sylius_address AS shipadress  ON cmd.shipping_address_id = shipadress.id WHERE cmd.checkout_state = "completed" AND cmd.shipping_state != "shipped" AND date(cmd.checkout_completed_at) ="{date_cmd_non_livre}";'

In [12]:
date_formated = datetime.strptime(date_cmd_non_livre, '%Y-%m-%d')
file_name = f"{date_formated.strftime('%Y%m%d')}_tableau_commandes_non_livres.csv"
save_data(comande_non_livre_utilisateur_info,file_name)

<ipython-input-4-993b066a6deb>:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_dataFrame = pd.read_sql(query,mydb)


# Toutes les commandes livrees

Fournir toutes les commandes livrées selon la date fournie

In [13]:
date_cmd_livre = "2023-07-17"

In [14]:
query_all_shipped_order = 'SELECT cmd.id, cmd.token_value,cmd.number AS numero,cmd.checkout_completed_at AS Date_finalisation_commande,cmd.items_total,cmd.adjustments_total AS Frais_livraison, cmd.total,billingadress.first_name AS Facturation_first_name,billingadress.last_name AS Facturation_last_name,billingadress.phone_number as Facturation_phone_number, shipadress.first_name AS Livraison_first_name,shipadress.last_name AS Livraison_last_name,shipadress.phone_number as Livraison_phone_number,cmd.shipping_state FROM andaal_db.sylius_order AS cmd JOIN andaal_db.sylius_address AS billingadress  ON cmd.billing_address_id = billingadress.id  JOIN andaal_db.sylius_address AS shipadress  ON cmd.shipping_address_id = shipadress.id WHERE cmd.checkout_state = "completed" AND cmd.shipping_state = "shipped" AND date(cmd.checkout_completed_at) = "{date_cmd_livre}";'
#AND date(cmd.checkout_completed_at) = {date_cmd_livre}

In [15]:
date_formated = datetime.strptime(date_cmd_livre, '%Y-%m-%d')
file_name = f"{date_formated.strftime('%Y%m%d')}_tableau_commandes_livres.csv"
save_data(query_all_shipped_order,file_name)

<ipython-input-4-993b066a6deb>:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_dataFrame = pd.read_sql(query,mydb)


# Nombre de livres vendus

Lorsqu'une date est fourni, il faudra extraire le nombre de livres vendu et non livré. Ceci est pour suivre les
commandes non livrées.

In [16]:
date_livre_vendu = "2023-06-12 20:29:21"
date_debut = date_livre_vendu[0:10]+" 00:00:00"#utilisé pour choisir sur une plage d'heure:si on veut pour aujourd'hui à 16h on prend d'aujourd'hui à 00:00 à 16h
code_livre = "ca"#utilisé pour différencier un livre d'un autre manuel scolaire

Requete pour avoir tous les livres vendus et non livrés

In [17]:
query_livres_vendu ='SELECT livre.product_name, SUM(livre.quantity) AS total_quantity FROM andaal_db.sylius_order AS cmd JOIN andaal_db.sylius_order_item AS livre ON cmd.id = livre.order_id WHERE (cmd.payment_state = "paid" OR (cmd.notes LIKE "%status: successful%" AND cmd.payment_state = "awaiting_payment" AND cmd.checkout_state = "completed") AND cmd.shipping_state <> "shipped" AND (checkout_completed_at BETWEEN  "date_debut" AND "{date_livre_vendu}")) AND livre.variant_id IN (SELECT id FROM andaal_db.sylius_product_variant WHERE code LIKE CONCAT("{code_livre}","%")) GROUP BY livre.product_name;'

Exécution de la requete

In [18]:
file_name = date_livre_vendu+"_tableau_livre_vendu.csv"
save_data(query_livres_vendu,file_name)

<ipython-input-4-993b066a6deb>:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_dataFrame = pd.read_sql(query,mydb)


# Nombre de cahiers vendus

Lorsqu'une date est fournie, il faudra extraire le nombre de cahiers vendu et non livré. Ceci est pour suivre les
commandes non livrées.

In [19]:
date_cahier_vendu = "2023-06-12 20:29:21"
date_debut = date_cahier_vendu[0:10]+" 00:00:00"#utilisé pour choisir sur une plage d'heure:si on veut pour aujourd'hui à 16h on prend d'aujourd'hui à 00:00 à 16h
code_cahier = ""#utilisé pour différencier un livre d'un autre manuel scolaire

Requete pour avoir tous les cahiers vendus et non livrés

In [20]:
query_cahiers_vendu ='SELECT cahier.product_name, SUM(cahier.quantity) AS total_quantity FROM andaal_db.sylius_order AS cmd JOIN andaal_db.sylius_order_item AS cahier ON cmd.id = cahier.order_id WHERE (cmd.payment_state = "paid" OR (cmd.notes LIKE "%status: successful%" AND cmd.payment_state = "awaiting_payment" AND cmd.checkout_state = "completed") AND cmd.shipping_state <> "shipped" AND (checkout_completed_at BETWEEN  "date_debut" AND "{date_cahier_vendu}")) AND cahier.variant_id IN (SELECT id FROM andaal_db.sylius_product_variant WHERE code LIKE CONCAT("{code_cahier}","%")) GROUP BY cahier.product_name;'

Exécution de la requete

In [21]:
file_name = date_cahier_vendu+"_tableau_cahier_vendu.csv"
save_data(query_cahiers_vendu,file_name)

<ipython-input-4-993b066a6deb>:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_dataFrame = pd.read_sql(query,mydb)


# Nombre des autres fournitures vendus

Lorsqu'une date est fournie, il faudra extraire le nombre des autres fournitures vendu et non livré. Ceci est pour suivre les commandes non livrées.

In [22]:
date_autre_fourniture_vendu = "2023-06-12 20:29:21"
date_debut = date_autre_fourniture_vendu[0:10]+" 00:00:00"
code_autre_fourniture = ""

Requete pour avoir tous les cahiers vendus et non livrés

In [23]:
query_autre_fourniture_vendu ='SELECT autre_fourniture.product_name, SUM(autre_fourniture.quantity) AS total_quantity FROM andaal_db.sylius_order AS cmd JOIN andaal_db.sylius_order_item AS autre_fourniture ON cmd.id = autre_fourniture.order_id WHERE (cmd.payment_state = "paid" OR (cmd.notes LIKE "%status: successful%" AND cmd.payment_state = "awaiting_payment" AND cmd.checkout_state = "completed") AND cmd.shipping_state <> "shipped" AND (checkout_completed_at BETWEEN  "date_debut" AND "{date_autre_fourniture_vendu}")) AND autre_fourniture.variant_id IN (SELECT id FROM andaal_db.sylius_product_variant WHERE code LIKE CONCAT("{code_autre_fourniture}","%")) GROUP BY autre_fourniture.product_name;'

Exécution de la requête

In [24]:
file_name = date_autre_fourniture_vendu+"_tableau_autre_fourniture_vendu.csv"
save_data(query_autre_fourniture_vendu,file_name)

<ipython-input-4-993b066a6deb>:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_dataFrame = pd.read_sql(query,mydb)


## Nombre d'oeuvres vendues

Lorsqu'une date est fournie, il faudra extraire le nombre des autres fournitures vendu et non livré. Ceci est pour suivre les commandes non livrées.

In [25]:
date_oeuvre_vendu = "2023-06-12 20:29:21"
date_debut = date_oeuvre_vendu[0:10]+" 00:00:00"
code_oeuvre = ""

Requête pour avoir les oeuvres vendues et non livrés

In [26]:
query_oeuvre_vendu ='SELECT oeuvre.product_name, SUM(oeuvre.quantity) AS total_quantity FROM andaal_db.sylius_order AS cmd JOIN andaal_db.sylius_order_item AS oeuvre ON cmd.id = oeuvre.order_id WHERE (cmd.payment_state = "paid" OR (cmd.notes LIKE "%status: successful%" AND cmd.payment_state = "awaiting_payment" AND cmd.checkout_state = "completed") AND cmd.shipping_state <> "shipped" AND (checkout_completed_at BETWEEN  "date_debut" AND "{date_oeuvre_vendu}")) AND oeuvre.variant_id IN (SELECT id FROM andaal_db.sylius_product_variant WHERE code LIKE CONCAT("{code_oeuvre}","%")) GROUP BY oeuvre.product_name;'

Exécution de la requête

In [27]:
file_name = date_oeuvre_vendu+"_tableau_oeuvre_vendu.csv"
save_data(query_oeuvre_vendu,file_name)

<ipython-input-4-993b066a6deb>:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_dataFrame = pd.read_sql(query,mydb)
